<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Deep-Deterministic-Policy-Gradients-(DDPG)" data-toc-modified-id="Deep-Deterministic-Policy-Gradients-(DDPG)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Deep Deterministic Policy Gradients (DDPG)</a></span><ul class="toc-item"><li><span><a href="#Import-the-Necessary-Packages" data-toc-modified-id="Import-the-Necessary-Packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import the Necessary Packages</a></span></li><li><span><a href="#Instantiate-the-Environment-and-Agent" data-toc-modified-id="Instantiate-the-Environment-and-Agent-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Instantiate the Environment and Agent</a></span><ul class="toc-item"><li><span><a href="#Examine-the-state-and-action-spaces" data-toc-modified-id="Examine-the-state-and-action-spaces-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Examine the state and action spaces</a></span></li></ul></li><li><span><a href="#Train-the-Agent-with-DDPG" data-toc-modified-id="Train-the-Agent-with-DDPG-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Train the Agent with DDPG</a></span><ul class="toc-item"><li><span><a href="#4.-Watch-a-Smart-Agent!" data-toc-modified-id="4.-Watch-a-Smart-Agent!-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>4. Watch a Smart Agent!</a></span></li><li><span><a href="#5.-Explore" data-toc-modified-id="5.-Explore-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>5. Explore</a></span></li></ul></li></ul></li></ul></div>

# Deep Deterministic Policy Gradients (DDPG)
---
In this notebook, we train DDPG with OpenAI Gym's BipedalWalker-v2 environment.

## Import the Necessary Packages

In [1]:
from unityagents import UnityEnvironment
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

## Instantiate the Environment and Agent

In [2]:
env = UnityEnvironment(file_name='Reacher_Windows_x86_64/Reacher.exe')

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Creating functions for common environment processes

In [3]:
def environment_step(env, action, brain_name):
    env_info = env.step(action)[brain_name]
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0] 
    
    return next_state, reward, done, None

def environment_reset(env, brain_name, train_mode=True):
    env_info = env.reset(train_mode=train_mode)[brain_name] # reset the environment
    
    return env_info

### Examine the state and action spaces

In [4]:
# reset the environment
env_info = environment_reset(env, brain_name)

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [5]:
states

array([[ 0.00000000e+00, -4.00000000e+00,  0.00000000e+00,
         1.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -4.37113883e-08,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00, -1.00000000e+01,
         0.00000000e+00,  1.00000000e+00, -0.00000000e+00,
        -0.00000000e+00, -4.37113883e-08,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  5.75471878e+00,
        -1.00000000e+00,  5.55726671e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -1.68164849e-01]])

## Train the Agent with DDPG

Run the code cell below to train the agent from scratch.  Alternatively, you can skip to the next code cell to load the pre-trained weights from file.

In [6]:
agent = Agent(state_size=33, action_size=4, random_seed=42)

In [7]:
def ddpg(n_episodes=2000, max_t=2000):
    scores_deque = deque(maxlen=100)
    scores = []
    max_score = -np.Inf
    for i_episode in range(1, n_episodes+1):
        env_info = environment_reset(env, brain_name)
        state = env_info.vector_observations[0]
        agent.reset()
        score = 0
        count = 0
        for t in range(max_t):
            action = agent.act(state)
            next_state, reward, done, _ = environment_step(env, action, brain_name)
            #next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            count = t
            if done:
                break 
        scores_deque.append(score)
        scores.append(score)
        print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f} Episode timesteps: {}'.format(i_episode, np.mean(scores_deque), score, count), end="")
        if i_episode % 100 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
        if np.mean(scores_deque)>=30.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

scores = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

Episode 100	Average Score: 0.90	Score: 1.01 Episode timesteps: 1000
Episode 200	Average Score: 0.92	Score: 0.46 Episode timesteps: 1000
Episode 300	Average Score: 0.97	Score: 1.26 Episode timesteps: 1000
Episode 400	Average Score: 0.80	Score: 0.53 Episode timesteps: 1000
Episode 500	Average Score: 0.87	Score: 0.96 Episode timesteps: 1000
Episode 600	Average Score: 0.94	Score: 0.94 Episode timesteps: 1000
Episode 700	Average Score: 0.96	Score: 0.75 Episode timesteps: 1000
Episode 800	Average Score: 0.89	Score: 1.70 Episode timesteps: 1000
Episode 900	Average Score: 0.92	Score: 1.15 Episode timesteps: 1000
Episode 1000	Average Score: 0.87	Score: 2.06 Episode timesteps: 1000
Episode 1100	Average Score: 0.88	Score: 1.54 Episode timesteps: 1000
Episode 1200	Average Score: 0.92	Score: 0.90 Episode timesteps: 1000
Episode 1300	Average Score: 0.86	Score: 0.86 Episode timesteps: 1000
Episode 1400	Average Score: 0.91	Score: 0.27 Episode timesteps: 1000
Episode 1500	Average Score: 0.94	Score: 0.6

KeyboardInterrupt: 

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [ ]:
agent.actor_local.load_state_dict(torch.load('checkpoint_actor.pth'))
agent.critic_local.load_state_dict(torch.load('checkpoint_critic.pth'))

state = env.reset()
agent.reset()   
while True:
    action = agent.act(state)
    env.render()
    next_state, reward, done, _ = env.step(action)
    state = next_state
    if done:
        break
        
env.close()

### 5. Explore

In this exercise, we have provided a sample DDPG agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster than this benchmark implementation.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task!
- Write your own DDPG implementation.  Use this code as reference only when needed -- try as much as you can to write your own algorithm from scratch.
- You may also like to implement prioritized experience replay, to see if it speeds learning.  
- The current implementation adds Ornsetein-Uhlenbeck noise to the action space.  However, it has [been shown](https://blog.openai.com/better-exploration-with-parameter-noise/) that adding noise to the parameters of the neural network policy can improve performance.  Make this change to the code, to verify it for yourself!
- Write a blog post explaining the intuition behind the DDPG algorithm and demonstrating how to use it to solve an RL environment of your choosing.  